In [1]:
import numpy as np
import random
import itertools
import math
import tqdm
from tqdm import trange
import time

# Helper Function

In [2]:
key_mapping={
    'C':0,
    'D':2,
    'E':4,
    'F':5,
    'G':7,
    'A':9,
    'B':11
}
def key2num(key):  
  key=key.upper()
  num=key_mapping[key[0]]
  modifier=len(key)
  if modifier==1:
    return num
  elif key[1]=='#':
    return (num+(modifier-1))%12
  elif key[1]=='B' or key[1]=='-':
    return (num-(modifier-1))%12
  elif key[1]=='X':
    return (num+(modifier-1)*2)%12

# key_list to number_list
def keys2num(keys):
    return [key2num(key) for key in keys]

In [3]:
# function to distingish whether notes in given timestamp and chord are within or outside the chord (0=outside,1=within)
import json
with open('../modules/json_files/keychorddict.json') as json_file:
    chord_notes = json.load(json_file)
  
    def note_2_class(chord,notes_at_t,chord_notes=chord_notes):
        note_in_chord=chord_notes[chord]['idx']
        return [int(note in note_in_chord) for note in notes_at_t]

In [151]:
h_states.index('MajorI')

17

In [152]:
test.likelihood(1 ,[0, 3, 7], 'cMinor') 

0.0694776306723782

In [154]:
test.forward_table=np.empty((2,test.no_of_state,2))
test.forward_table[:]= np.NaN
test.key=['CMajor','CMajor']
[test.forward(1,i,[[0,4,7],[0,3,7]],True) for i in range(len(h_states))]

[(3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (0.00011625644770959873, 17.0),
 (1.088906795304897e-15, 17.0),
 (3.048939026853712e-14, 17.0),
 (3.4300564052104254e-21, 17.0),
 (9.800161157744072e-15, 17.0),
 (5.008971258402526e-14, 17.0),
 (0.00029872703080263387, 17.0),
 (4.86433437997341e-12, 17.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-21, 17.0),
 (1.3066881543658765e-14, 17.0),
 (3.4300564052104254e-21, 17.0),
 (3.4300564052104254e-2

# HMM

In [144]:
### I threat the total length of time as the length of the observered list

class HMM:
    def __init__(self,no_of_state,no_of_value,states,values):
        #randomize all matrix, each row should sum to 1
        #self.emssion_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_value) for i in range(no_of_state)])]) #b[i][O]  --> probability to emit values[O] from states[i]
        

        #Modification: 2 parameter for all chord  [note inside chord & note outside chord]
        self.zero=0.00000000001
        self.emssion_matrix=np.random.rand(no_of_value)
        self.emssion_matrix=self.emssion_matrix/self.emssion_matrix.sum()
        if self.emssion_matrix[0]>self.emssion_matrix[1]:
            temp=self.emssion_matrix[0]
            self.emssion_matrix[0]=self.emssion_matrix[1]
            self.emssion_matrix[1]=temp
        self.emssion_matrix=np.array([self.emssion_matrix,]*len(states)) 
        self.emssion_matrix=np.array([[0.2,0.8],]*len(states)) 
        # assume same probability to emit "note within chord"  for all chords
        
        self.initial_matrix= np.random.rand(no_of_state) #π[i] --> probability to start at states[i]
        self.initial_matrix/= self.initial_matrix.sum()
        
        self.transition_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_state) for i in range(no_of_state)])])  #a[i][j] --> probability from states[i] transit to states[j]
        self.T1=None
        self.T2=None
        self.no_of_state=no_of_state
        self.states=states
        self.values=values
        self.observered=None
        self.key=None
        self.t=None
        self.MinorMask=np.array([1, 1,
          1, 1, 1,
          1,
          1, 1,
          1, 1, 1,
          1, 1,1,1,
          1, 1,
          0,
          0,0,0,
          0,
          0,
          0,0,
          0,0,0,0,0,
          0,0,0
        ])
        self.MajorMask=(self.MinorMask*-1)+1
        
        self.probit_at_i_table=None
        self.probit_transit_i_j_table=None
        self.forward_table=None
        self.backward_table=None
        
        self.chord_probit=None
        
        

        #self.prev_initial_matrix=self.initial_matrix
        #self.prev_emssion_matrix=self.emssion_matrix
        #self.prev_transition_matrix=self.transition_matrix
  #  def update_state_name(self,name):
     #   self.states=name
        
    def debug(self):
        print('initial_matrix\n',self.initial_matrix)
        print('transition_matrix\n',self.transition_matrix)
        print('emission_matrix\n',self.emssion_matrix)
    
    def likelihood(self,state,ob_t,key):
        index_to_pitch_sharp = {
            0: "C",
            1: "C#",
            2: "D",
            3: "D#",
            4: "E",
            5: "F",
            6: "F#",
            7: "G",
            8: "G#",
            9: "A",
            10: "A#",
            11: "B",
        }
        if key[-5:]!=self.states[state][:5]:
            return self.zero
        chord=key+self.states[state][5:]
        
        num2note=[index_to_pitch_sharp[e] for e in ob_t]

        a=noteToChord.NoteToChord(num2note,key,5,0)

        score = np.array([i['Score'] for i in a])
        score=score-min(score)
        score/=score.sum()
        chords=[i['Chord'] for i in a]

        try:
            idx=chords.index(chord)
        except ValueError:
            idx=None
            
        
            
        if idx==None:
            return self.zero

        ob_t=note_2_class(chord,ob_t) #2 class
        
        prob=1
        for x in ob_t:
            prob*=(self.emssion_matrix[state][x])
        return score[idx]/score.sum()   *prob#/self.chord_probit[state] # assume uniform probability
    """#original function
    def likelihood(self,state,ob_t):
        #Modification: convert observation[t] from notes to 2_classes
        chord=self.states[state]  #numeric to chord name
        ob_t=note_2_class(chord,ob_t) #2 class
        
        prob=1
        for x in ob_t:
            prob*=(self.emssion_matrix[state][x])
        return prob
    """
 
    def forward(self,t,j,ob=None,mode=False):
        self.t=t
        if ob is None:
            ob=self.observered
        if np.isnan(self.forward_table[t][j]).any():
            if t==0:
                if mode==True:
                    if 'Major' in self.key[t]:
                        self.forward_table[t][j]=[(self.initial_matrix*self.MajorMask)[j]*self.likelihood(j,ob[t],self.key[t]),0]
                    else:
                        self.forward_table[t][j]=[(self.initial_matrix*self.MinorMask)[j]*self.likelihood(j,ob[t],self.key[t]),0]
                    return self.forward_table[t][j][0],self.forward_table[t][j][1]
                else:
                    self.forward_table[t][j]=self.initial_matrix[j]*self.likelihood(j,ob[t])
                    return self.forward_table[t][j]
            else:          
                if mode==True:

                    if self.key[t-1][:5]==self.key[t][:5]:
            
                        if 'Major' in self.key[t]:
                            result=np.array([self.forward(t-1,i,ob,mode)[0]*self.transition_matrix[i][j] for i in range(self.no_of_state)])*self.MajorMask* self.likelihood(j,ob[t],self.key[t])
                        else:
                            result=np.array([self.forward(t-1,i,ob,mode)[0]*self.transition_matrix[i][j] for i in range(self.no_of_state)])*self.MinorMask * self.likelihood(j,ob[t],self.key[t])
          
                    else:
        
                        if 'Major' in self.keupp[t]:        
                            result=self.MajorMask * self.likelihood(j,ob[t],self.key[t])
                        else:
                            result=self.MinorMask * self.likelihood(j,ob[t],self.key[t])
    
        
                    self.forward_table[t][j]=[np.max(result),np.argmax(result)]

                    return self.forward_table[t][j][0],self.forward_table[t][j][1]
                else:
                    self.forward_table[t][j]=sum([self.forward(t-1,i,ob,mode)*self.transition_matrix[i][j] for i in range(self.no_of_state)]) * self.likelihood(j,ob[t])
                    return self.forward_table[t][j]
        else:
            return self.forward_table[t][j]

    def backward(self,t,i,ob=None,mode=False):
        if ob is None:
            ob=self.observered
        if np.isnan(self.backward_table[t][i]) or mode==True:
            if t==len(ob)-1:
                self.backward_table[t][i]=1
                return self.backward_table[t][i]
            else:
                if mode==True:
                    self.backward_table[t][i]=max([self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob,mode) for j in range(self.no_of_state)])
                    return self.backward_table[t][i]
                else:
                    self.backward_table[t][i]=sum([self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob,mode) for j in range(self.no_of_state)])
                    return self.backward_table[t][i]
        else:
            return self.backward_table[t][i]
            
           
    def probit_at_i(self,t,i,ob=None):#Gamma γt(i) = P(qt = i|O,λ)      
        if ob is None:
            ob=self.observered
        if np.isnan(self.probit_at_i_table[t][i]):
            numerator=self.forward(t,i,ob)*self.backward(t,i,ob)#sum probability of all path passing through state[i] at time t
            denominator=sum([self.forward(t,j,ob)*self.backward(t,j,ob) for j in range(self.no_of_state)]) #prob of passing through  ALL_state at time t
            self.probit_at_i_table[t][i]=numerator/denominator
            return self.probit_at_i_table[t][i]
        else:
            return self.probit_at_i_table[t][i]
    
    def probit_transit_i_j(self,t,i,j,ob=None):#epsilon ξt(i, j) = P(qt = i,qt+1 = j|O,λ)
        if ob is None:
            ob=self.observered
        if np.isnan(self.probit_transit_i_j_table[t][i][j]):
            numerator=self.forward(t,i,ob)*self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob)#sum probability of all path transit from state[i] to state[j] at time t
            denominator=sum([sum([self.forward(t,m,ob)*self.transition_matrix[m][n]*self.likelihood(n,ob[t+1])*self.backward(t+1,n,ob) for n in range(self.no_of_state)]) for m in range(self.no_of_state)]) #prob of ALL transition combination at time t
            self.probit_transit_i_j_table[t][i][j]=(numerator/denominator)
            return self.probit_transit_i_j_table[t][i][j]
        else:
            return self.probit_transit_i_j_table[t][i][j]
        
    #modify from https://stackoverflow.com/questions/9729968/python-implementation-of-viterbi-algorithm/9730083 , author RBF06(https://stackoverflow.com/users/3311728/rbf06)
    def predict(self,ob,test_key):  
        self.forward_table=np.empty((len(ob),self.no_of_state,2))
        self.forward_table[:]= np.NaN
        self.key=test_key

        T1=np.empty((self.no_of_state,len(ob)),'d')
        T2=np.empty((self.no_of_state,len(ob)),'B')
        for idx in range(self.no_of_state):
            T1[idx,0]=self.forward(0,idx,ob,True)[0]
        T2[:,0]=0
        self.T1=T1
        
        for i in range(1,len(ob)):
            for idx in range(self.no_of_state):
                T1[idx,i],T2[idx,i]=self.forward(i,idx,ob,True)

        x = np.empty(len(ob), 'B')
        x[-1] = np.argmax(T1[:, len(ob) - 1])
        
        self.T2=x
        for i in reversed(range(1, len(ob))):
            x[i - 1] = T2[x[i], i]
        return x
    
    def get_detail(self,label):
        key='Major' if 'M' in label else 'Minor'
        chord=label.split("_")[1]
        key_note=label.split('M')[0] if key=='Major' else label.split('m')[0].lower()
        if key=='Major':
            if key_note=='F#':
                key_note='Gb'
            elif key_note=='A#':
                key_note='Bb'
            elif key_note=='C#':
                key_note='Db'
            elif key_note=='G#':
                key_note='Ab'
            elif key_note=='D#':
                key_note='Eb'
        else:
            if key_note=='gb':
                key_note='f#'
            elif key_note=='db':
                key_note='c#'
            elif key_note=='ab':
                key_note='g#'
            elif key_note=='d#':
                key_note='eb'
                
        if 'German' in chord:
            chord=key+'GerVI'
        elif 'Dim7' in chord:
            chord=key+'DimVII7'
        elif 'FrenchVI' in chord:
            chord=key+'FreVI'
        else:
            chord=key+chord
        return chord,key_note
    
    def train_supervisied(self,obs,labels): # by MLE
        initial_matrix= np.zeros(self.no_of_state)
        emssion_matrix=np.zeros((2)) 
        transition_matrix=np.array([np.zeros(self.no_of_state) for i in range(self.no_of_state)])

        for idx_1,label in enumerate(labels):
        #    self.update_state_name([key_name[idx_1].lower()[:-1]+states for states in h_states])
            for idx_2,lab in enumerate(label):
                Pre_chord,Pre_key_name=self.get_detail(label[idx_2-1])
                chord,key_name=self.get_detail(label[idx_2])
                if idx_2==0:
                    initial_matrix[self.states.index(chord)]+=1
                else:
                    if Pre_chord[:5]==chord[:5]: ## ignore key change
                        transition_matrix[self.states.index(Pre_chord)][self.states.index(chord)]+=1
                        #note_2_class(chord,notes_at_t,chord_notes=chord_notes):
                    ob_t=note_2_class(key_name+chord,obs[idx_1][idx_2]) #2 class
                    for x in ob_t:
                        emssion_matrix[x]+=1
                        
        #save back to model

        self.initial_matrix=np.array([item/initial_matrix.sum() if item >0 else 0.00000001 for item in initial_matrix])
        self.transition_matrix=np.array([row/row.sum() if row.sum()>0 else row+0.00000001 for row in transition_matrix])
        for row in range(self.transition_matrix.shape[0]):
            for col in range(self.transition_matrix.shape[1]):
                if self.transition_matrix[row][col]==0:
                    self.transition_matrix[row][col]=0.00000001
        self.emssion_matrix=np.array([emssion_matrix/emssion_matrix.sum()] *self.no_of_state) 

        
"""        
    def train(self,obs,key_name,epochs=2):
        #O:observed values
        #λ:model parameters

        
        for epoch in range(epochs):
            for ob_idx,ob in enumerate(obs):
                self.update_state_name([key_name[ob_idx].lower()[:-1]+states for states in h_states])
                print('running',ob_idx,key_name[ob_idx].lower()[:-1])
                self.probit_at_i_table=np.empty((len(ob),self.no_of_state))
                self.probit_transit_i_j_table=np.empty((len(ob),self.no_of_state,self.no_of_state))
                self.forward_table=np.empty((len(ob),self.no_of_state))
                self.backward_table=np.empty((len(ob),self.no_of_state))
                self.probit_at_i_table[:]= np.NaN
                self.probit_transit_i_j_table[:]=np.NaN
                self.forward_table[:]= np.NaN
                self.backward_table[:]=np.NaN
                self.chord_probit=self.transition_matrix.sum(axis=0)
                self.chord_probit/=self.chord_probit.sum()
                #initalize DP table
                for t, i in tqdm.tqdm(itertools.product(range(len(ob)),range(self.no_of_state))):
                    self.forward(t,i,ob)
                    self.backward(t,i,ob)
                    self.probit_at_i(t,i,ob)
                    for j in range(self.no_of_state):
                        if t!=len(ob)-1:
                            self.probit_transit_i_j(t,i,j,ob)
                
                
                
                #initial matrix
                for i in range(self.no_of_state):
                    self.initial_matrix[i]=self.probit_at_i(0,i,ob)
                    if self.initial_matrix[i]==0:
                        self.initial_matrix[i]=0.00000001
                #transition matrix
                for i, j in itertools.product(range(self.no_of_state),range(self.no_of_state)):
                    self.transition_matrix[i][j]=sum([self.probit_transit_i_j(t,i,j,ob) for t in range(len(ob)-1)])/sum([self.probit_at_i(t,i,ob) for t in range(len(ob)-1)])
                    if self.transition_matrix[i][j]==0 :
                        self.transition_matrix[i][j]=0.00000001
                #emission matrix
                for j, k in itertools.product(range(self.no_of_state),range(len(self.values))):   
                    total=0
                    
                    #Modification: convert notes to 2 classes (outside or inside given chord)
                    chord=self.states[j]  #numeric to chord name
                    ob_2_class=[note_2_class(chord,ob_t) for ob_t in ob] #2 class

                    for t in range(len(ob)):
                        if k in ob_2_class[t]:
                            #Modification: multiple by how many times do k appear at timestamp t
                            total+=self.probit_at_i(t,j,ob)*ob_2_class[t].count(k)  
                            
                    #Modification: multiple by len(ob[t]), which is the total length of notes at timestamp t                    
                    self.emssion_matrix[:,k]=total/sum([self.probit_at_i(t,j,ob)*len(ob[t]) for t in range(len(ob))])  
                    
                    #smoothing
                    
                    if self.emssion_matrix[j][k]==0 :
                        self.emssion_matrix[:,k]=0.00000001
"""                        

"        \n    def train(self,obs,key_name,epochs=2):\n        #O:observed values\n        #λ:model parameters\n\n        \n        for epoch in range(epochs):\n            for ob_idx,ob in enumerate(obs):\n                self.update_state_name([key_name[ob_idx].lower()[:-1]+states for states in h_states])\n                print('running',ob_idx,key_name[ob_idx].lower()[:-1])\n                self.probit_at_i_table=np.empty((len(ob),self.no_of_state))\n                self.probit_transit_i_j_table=np.empty((len(ob),self.no_of_state,self.no_of_state))\n                self.forward_table=np.empty((len(ob),self.no_of_state))\n                self.backward_table=np.empty((len(ob),self.no_of_state))\n                self.probit_at_i_table[:]= np.NaN\n                self.probit_transit_i_j_table[:]=np.NaN\n                self.forward_table[:]= np.NaN\n                self.backward_table[:]=np.NaN\n                self.chord_probit=self.transition_matrix.sum(axis=0)\n                self.c

# Training

In [164]:
import json
with open('../data/training_data.json') as json_file:
    data = json.load(json_file)

In [165]:
h_states=['MinorI', 'MinorI+',
          'MinorbII', 'MinorII', 'MinorII7',
          'MinorIII',
          'MinorIV', 'MinorIV+',
          'MinorV', 'MinorV+', 'MinorV+7',
          'MinorVI', 'MinorGerVI', 'MinorFreVI', 'MinorItaVI',
          'MinorVII', 'MinorDimVII7',#, 'MinorDimVII'
          'MajorI',
          'MajorbII','MajorII','MajorII7',
          'MajorIII',
          'MajorIV',
          'MajorV','MajorV7',
          'MajorbVI','MajorGerVI','MajorFreVI','MajorItaVI','MajorVI',
          'MajorVII','MajorVII7','MajorDimVII7'
]
#all possible chordm will add the key to the chord afterward e.g. cMinorI

In [166]:
score=np.array(list(data.keys()))
np.random.shuffle(score)
test_score=score[-3:]
score=score[:-3]
test_score,score

(array(['notestochord.mxl', 'Nocturne_in_F_Minor.mxl',
        'Prelude_in_B_Minor.mxl'], dtype='<U35'),
 array(['Prelude_in_F_Major.mxl', 'Menuet_in_G_Minor.mxl',
        'Nocturne_in_E_Major.mxl', 'Prelude_in_B_Minor_Opus104a.mxl',
        'Etude_in_Gb_Major_Opus_25.mxl', 'Nocturne_in_C_Minor.mxl',
        'Etude_in_F_Major.mxl', 'Prelude_in_Db_Major.mxl',
        'Moonlight_Sonata_1st_Movement.mxl', 'Etude_in_Gb_Major.mxl',
        'Piano_Sonata_No._11.mxl', 'Il_Vecchio_Castello.mxl',
        'Prelude_in_G_Major.mxl', 'Etude_in_C_Minor.mxl',
        'Nocturne_No._20_in_C_Minor.mxl',
        'Prlude_Opus_28_No._4_in_E_Minor.mxl', 'Nocturne_in_E_Minor.mxl',
        'Prelude_in_A_Major.mxl', 'Nocturne_in_F_Major.mxl',
        'Etude_in_F_Minor.mxl', 'Minuet_in_G_Major_2nd.mxl',
        'Prelude_in_B_Major.mxl', 'Prelude_in_C_Minor.mxl',
        'Minuet_in_F.mxl', 'Nocturne_in_Eb_Major.mxl',
        'Nocturne_in_B_Major.mxl'], dtype='<U35'))

In [167]:
#prepare train data
score_name=score


x_train=[]
y_train=[]
for idx_root,name in enumerate(score_name):
    select_idx=[]
    #only extract chord in the same key with the score
    for idx,chord in enumerate(data[name]['chord_seq']):
      #  if chord.startswith(score_key[idx_root]):
            select_idx.append(idx)
    training=[]
    training_label=[]
    for idx in select_idx:
        training.append(data[name]['note_seq'][idx])
  #      tail=data[name+'.mxl']['chord_seq'][idx].split('_')[1]
        tail=data[name]['chord_seq'][idx]
        #if tail.startswith('German'):
            #tail='Ger'+str(tail[6:])
        #elif tail=='Dim7':
            #tail='DimVII'
        #training_label.append(score_key[idx_root][:-1].lower()+'Minor'+tail)
        training_label.append(tail)
    training=[keys2num(segment)for segment in training]
    
    x_train.append(training)
    y_train.append(training_label)

In [168]:
test=HMM(len(h_states),2,h_states,["outside chord","inside chord"])

In [169]:
test.train_supervisied(x_train,y_train)#initialize parameter

In [170]:
test.debug()

initial_matrix
 [4.61538462e-01 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08 3.46153846e-01 1.00000000e-08 3.84615385e-02
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.53846154e-01
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
 1.00000000e-08]
transition_matrix
 [[2.20385675e-02 3.30578512e-02 1.65289256e-02 ... 1.00000000e-08
  1.00000000e-08 1.00000000e-08]
 [1.39534884e-01 4.65116279e-02 6.97674419e-02 ... 1.00000000e-08
  1.00000000e-08 1.00000000e-08]
 [2.50000000e-01 4.16666667e-02 1.00000000e-08 ... 1.00000000e-08
  1.00000000e-08 1.00000000e-08]
 ...
 [1.00000000e-08 1.00000000e-08 1.00000000e-08 ... 1.00000000e-08
  1.00000000e-08 1.00000000e-08]
 [1.00000000e-08 1.00000000e-08 1.00000000e-08 ... 1.000000

# Evaluation

In [204]:
test_key=[]
for e in testing_label:
    temp_key=e.split('_')[0]
    temp_name=temp_key[:-1]
    temp_key=temp_key[-1]
    if temp_key=='m':
        test_key.append(temp_name.lower()+'Minor')
    else:
        test_key.append(temp_name[0].upper()+temp_name[1:].lower()+'Major')        

In [205]:
#prepare test data
test_score=test_score
#prepare the test data for a specific score (select by index)
score_test_choice=0


#only extract those chords that are in te same key with the score
test_sample=[]
select_idx=[]
testing_label=[]
name=test_score[score_test_choice]
for idx,chord in enumerate(data[name]['chord_seq']):
   # if chord.startswith(test_key[score_test_choice]):
        select_idx.append(idx)
training=[]
training_label=[]
for idx in select_idx:
    training.append(data[name]['note_seq'][idx])
    training_label.append(data[name]['chord_seq'][idx])
    
test_note=training
training=[keys2num(segment)for segment in training]
test_sample=training
testing_label=training_label


In [206]:



test_score_key=test_key[score_test_choice]
test_states=[test_score_key.lower()[:-1]+states for states in h_states]



In [207]:
len(test_key)

233

In [208]:
len(test_sample)

233

In [209]:
import sys
sys.path.append('../modules')
import noteToChord

In [210]:
prediction=test.predict(test_sample,test_key)

In [211]:
[h_states[i] for i in prediction[:10]]

['MajorVII',
 'MajorI',
 'MajorI',
 'MajorDimVII7',
 'MajorGerVI',
 'MajorVI',
 'MajorI',
 'MajorVII',
 'MajorI',
 'MajorDimVII7']

In [215]:
testing_label[:3],test_key[:3]

(['EbM_V', 'EbM_I', 'EbM_V'], ['EbMajor', 'EbMajor', 'EbMajor'])

In [213]:
test_sample[:3]

[[10, 2, 5, 8, 3], [10, 3, 7, 5], [10, 2, 5, 8, 7]]

In [219]:
h_states[np.argmax(np.array([test.likelihood(i ,[10, 2, 5, 8, 7], 'EbMajor')  for i in range(len(h_states))]))]

'MajorDimVII7'

In [224]:
num2note=[index_to_pitch_sharp[e] for e in [10, 3, 7, 5]]
print(num2note)
num2note=['B-','E-','G','F']
noteToChord.NoteToChord(num2note,'EbMajor',5,0)

['A#', 'D#', 'G', 'F']


[{'Chord': 'EbMajorI',
  'Score': 3809.0,
  'pitch match': 3,
  'name match': 3,
  'root present': True,
  'edit distance': 14,
  'length match': False},
 {'Chord': 'EbMajorV',
  'Score': 2727.0,
  'pitch match': 2,
  'name match': 2,
  'root present': True,
  'edit distance': 3,
  'length match': False},
 {'Chord': 'EbMajorV7',
  'Score': 2717.5714285714284,
  'pitch match': 2,
  'name match': 2,
  'root present': True,
  'edit distance': 6,
  'length match': True},
 {'Chord': 'EbMajorIII',
  'Score': 2703.75,
  'pitch match': 2,
  'name match': 2,
  'root present': True,
  'edit distance': 15,
  'length match': False},
 {'Chord': 'EbMajorII7',
  'Score': 2702.0,
  'pitch match': 2,
  'name match': 2,
  'root present': True,
  'edit distance': 19,
  'length match': True}]

In [221]:
index_to_pitch_sharp = {
            0: "C",
            1: "C#",
            2: "D",
            3: "D#",
            4: "E",
            5: "F",
            6: "F#",
            7: "G",
            8: "G#",
            9: "A",
            10: "A#",
            11: "B",
        }


In [741]:
h_states.index('MajorV')

23

In [742]:
h_states.index('MajorIV')

22

In [743]:
test.initial_matrix[17:]

array([1.00000000e+00, 1.00000000e-08, 3.84615385e-02, 1.00000000e-08,
       1.00000000e-08, 1.00000000e-08, 1.92307692e-01, 1.00000000e-08,
       1.00000000e-08, 1.00000000e-08, 1.00000000e-08, 1.00000000e-08,
       1.00000000e-08, 1.00000000e-08, 1.00000000e-08, 1.00000000e-08])

In [744]:
[test.states[e] for e in prediction][:10]

['MajorV',
 'MajorI',
 'MajorV',
 'MajorI',
 'MajorV',
 'MajorI',
 'MajorV',
 'MajorI',
 'MajorV',
 'MajorI']

In [607]:


#rephase prediction to share same format with testinng_label
prediction=[test_states[idx].split('Minor')[1] for idx in prediction]
testing_label=[label.split('_')[1] for label in testing_label]
for idx,lab in enumerate(testing_label):
    if lab.startswith('German'):
        testing_label[idx]='Ger'+str(lab[6:])



#evaluation
correct=0
wrong=0
for idx,label in enumerate(prediction):
    if label==testing_label[idx]:
        correct+=1
    else:
        wrong+=1
print('acc',correct/(correct+wrong),correct,wrong)

TypeError: list indices must be integers or slices, not str

In [608]:

note2chord_ans=[]
for notes in test_note:
    note2chord_ans.append(noteToChord.NoteToChord(list(notes.keys()),test_score_key[:-1]+'Minor',1,0)[0]['Chord'].split('Minor')[1])
#evaluation
correct=0
wrong=0
for idx,label in enumerate(note2chord_ans):
    if label==testing_label[idx]:
        correct+=1
    else:
        wrong+=1
print('acc',correct/(correct+wrong),correct,wrong)   

ValueError: not enough values to unpack (expected 7, got 0)

In [609]:
#remove 7 HMM
prediction=[each[:-1] if '7' in each else each for each in prediction]
testing_label=[each[:-1] if '7' in each else each for each in testing_label]
#evaluation
correct=0
wrong=0
for idx,label in enumerate(prediction):
    if label==testing_label[idx]:
        correct+=1
    else:
        wrong+=1
print('acc',correct/(correct+wrong),correct,wrong) 

acc 0.0 0 28


In [610]:
#remove 7 note2chord
note2chord_ans=[each[:-1] if '7' in each else each for each in note2chord_ans]
testing_label=[each[:-1] if '7' in each else each for each in testing_label]
#evaluation
correct=0
wrong=0
for idx,label in enumerate(note2chord_ans):
    if label==testing_label[idx]:
        correct+=1
    else:
        wrong+=1
print('acc',correct/(correct+wrong),correct,wrong) 

ZeroDivisionError: division by zero

# Debug

In [18]:
test.debug()

initial_matrix
 [1.e+00 1.e-08 1.e-08 1.e-08 1.e-08 1.e-08 1.e-08 1.e-08 1.e-08 1.e-08
 1.e-08 1.e-08 1.e-08 1.e-08 1.e-08 1.e-08 1.e-08 1.e-08]
transition_matrix
 [[1.00000000e-08 1.00000000e-08 8.33333333e-02 5.83333333e-01
  1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
  1.00000000e-08 2.50000000e-01 1.00000000e-08 8.33333333e-02
  1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
  1.00000000e-08 1.00000000e-08]
 [1.00000000e-08 1.00000000e-08 2.00000000e-01 1.00000000e-08
  1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
  1.00000000e-08 6.00000000e-01 1.00000000e-08 2.00000000e-01
  1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
  1.00000000e-08 1.00000000e-08]
 [1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
  1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
  1.00000000e-08 5.00000000e-01 5.00000000e-01 1.00000000e-08
  1.00000000e-08 1.00000000e-08 1.00000000e-08 1.00000000e-08
  1.00000000e-08 1.0000000

In [19]:
np.array(note2chord_ans)

array(['I', 'V', 'II', 'V+', 'VI', 'V+', 'GerVI', 'III', 'V+', 'I', 'V',
       'II', 'II', 'IV', 'VII', 'III', 'VI', 'I', 'IV', 'I', 'DimVII',
       'I', 'I', 'IV', 'III', 'II', 'I', 'I'], dtype='<U6')

In [20]:
np.array(prediction)

array(['I', 'V+', 'II', 'V+', 'VI', 'V+', 'I+', 'VI', 'V+', 'I+', 'V+',
       'IV', 'I+', 'bII', 'V+', 'I', 'VI', 'V+', 'I+', 'V+', 'DimVII',
       'I', 'V+', 'IV', 'I', 'II', 'V+', 'I'], dtype='<U6')

In [21]:
np.array(testing_label)

array(['I', 'V', 'VI', 'V', 'VI', 'VII', 'III', 'IV', 'V+', 'I', 'V',
       'VI', 'I+', 'IV', 'VII', 'I', 'VI', 'I+', 'IV', 'I', 'V+', 'I',
       'V+', 'IV', 'I', 'II', 'V+', 'I'], dtype='<U3')